Сопоставление математических выражений

In [ ]:
from kanren import run, var, fact
import kanren.assoccomm as la


# Определение математических операций
operation_add = 'сложение'
operation_mul = 'умножение'


# Объявление того, что операции коммутативные,
# посредством использования системы фактов

fact(la.commutative, operation_mul)
fact(la.commutative, operation_add)
fact(la.associative, operation_mul)
fact(la.associative, operation_add)


# Определение переменных
x, y, z = var('x'), var('y'), var('z')


# Генерация выражений
expr_base = (operation_add, (operation_mul, 5, -4), (operation_mul, (operation_add, 2, (operation_mul, 3, 5)), -2))
expr_test1 = (operation_add, (operation_mul, (operation_add, 2, (operation_mul, 3, x)), y), (operation_mul, 5, z))
expr_test2 = (operation_add, (operation_mul, z, 5), (operation_mul, y, (operation_add, (operation_mul, 3, x), 2)))
expr_test3 = (operation_add, (operation_add, (operation_mul, (operation_mul, 3, x), y), y), (operation_mul, 5, z))


# Сравнение выражений
print(run(0, (x, y, z), la.eq_assoccomm(expr_test1, expr_base)))
print(run(0, (x, y, z), la.eq_assoccomm(expr_test2, expr_base)))
print(run(0, (x, y, z), la.eq_assoccomm(expr_test3, expr_base)))

((5, -2, -4),)
((5, -2, -4),)
()


Проверка простых чисел

In [4]:
import itertools as it
import kanren.core as lc
from kanren import membero, isvar
from sympy.ntheory.generate import prime, isprime


# Проверка того, являются ли элементы n простыми числами
def verify_prime(n):
    if isvar(n):
        return lc.condeseq([(lc.eq, n, p)] for p in map(prime, it.count(1)))
    else:
        return lc.success if isprime(n) else lc.fail


# Объявление переменной
v = lc.var()


# Проверка того, является ли элемент списка простым числом
numbers_list = (31, 6, 35, 19, 17, 14, 25, 37, 5, 40)
print('\nПростые числа в списке:')
print(set(lc.run(0, v, (membero, v, numbers_list), (verify_prime, v))))


# Вывести первые 7 простых чисел
print('\nПервые 7 простых чисел:')
print(lc.run(7, v, verify_prime(v)))


Простые числа в списке:
{5, 37, 17, 19, 31}

Первые 7 простых чисел:
(2, 3, 5, 7, 11, 13, 17)


Парсинг генеалогического дерева

In [1]:
import json
from kanren import Relation, facts, run, conde, var, eq

# Проверка того, является ли 'х' родителем 'у'
def parent(x, y):
    return conde([father(x, y)], [mother(x, y)])

# Проверка того, является ли 'х' дедушкой (бабушкой) 'у'
def grandparent(x, y):
    temp = var()
    return conde((parent(x, temp), parent(temp, y)))

# Проверка братских (сестринских) отношений между 'а' и 'b'
def sibling(x, y):
    temp = var()
    return conde((parent(temp, x), parent(temp, y)))

# Проверка того, является ли х дядей у
def uncle(x, y):
    temp = var()
    return conde((father(temp, x), grandparent(temp, y)))

if __name__=='__main__':
    father = Relation()
    mother = Relation()

    with open('relationships.json', encoding='utf-8') as f:
        d = json.loads(f.read())

    for item in d['father']:
        facts(father, (list(item.keys())[0], list(item.values())[0]))

    for item in d['mother']:
        facts(mother, (list(item.keys())[0], list(item.values())[0]))

    x = var()

    # Дети Ивана
    name = 'Иван'
    output = run(0, x, father(name, x))
    print("\nСписок детей " + name + ":")
    for item in output:
        print(item)

    # Мать Никиты
    name = 'Никита'
    output = run(0, x, mother(x, name))[0]
    print("\nМать " + name + ":\n" + output)

    # Родители Виктора
    name = 'Виктор'
    output = run(0, x, parent(x, name))
    print("\nСписок родителей " + name + ":")
    for item in output:
        print(item)

    # Дедушка и бабушка Никиты
    name = 'Никита'
    output = run(0, x, grandparent(x, name))
    print("\nСписок бабушек и дедушек " + name + ":")
    for item in output:
        print(item)

    # Внуки и внучки Ольги
    name = 'Ольга'
    output = run(0, x, grandparent(name, x))
    print("\nСписок внуков " + name + ":")
    for item in output:
        print(item)

    # Братья и сестры Екатерины
    name = 'Екатерина'
    output = run(0, x, sibling(x, name))
    siblings = [x for x in output if x != name]
    print("\nСписок братьев и сестёр " + name + ":")
    for item in siblings:
        print(item)

    # Дяди Софии
    name = 'София'
    name_father = run(0, x, father(x, name))[0]
    output = run(0, x, uncle(x, name))
    output = [x for x in output if x != name_father]
    print("\nСписок дядей " + name + ":")
    for item in output:
        print(item)

    # Все супруги
    a, b, c = var(), var(), var()
    output = run(0, (a, b), father(a, c), mother(b, c))
    print("\nСписок всех супружеских пар:")
    for item in output:
        print('Муж:', item[0], '<==> Жена:', item[1])


Список детей Иван:
Михаил
Алексей
Дмитрий

Мать Никита:
Елена

Список родителей Виктор:
Алексей

Список бабушек и дедушек Никита:
Ольга
Иван

Список внуков Ольга:
София
Мария
Никита
Павел
Анна
Виктор
Сергей
Екатерина

Список братьев и сестёр Екатерина:
Мария
Павел
Виктор
Сергей

Список дядей София:
Алексей
Дмитрий

Список всех супружеских пар:
Муж: Иван <==> Жена: Ольга
Муж: Алексей <==> Жена: Татьяна
Муж: Дмитрий <==> Жена: Елена
Муж: Михаил <==> Жена: Надежда


Анализ географических данных

In [1]:
from kanren import run, fact, eq, Relation, var

border_rel = Relation()
seaside_rel = Relation()

input_seaside = 'coastal_states.txt'
input_border = 'adjacent_states.txt'

# Обработка файла с морскими штатами
with open(input_seaside, 'r') as f:
    text = f.read()
    seaside_set = text.split(',')

# Заполнение фактов о морских штатах
for loc in seaside_set:
    fact(seaside_rel, loc)

# Обработка файла с граничащими территориями
with open(input_border, 'r') as f:
    connections = [row.strip().split(',') for row in f if row and row[0].isalpha()]

# Запись отношений соседства
for group in connections:
    main_territory, surrounding = group[0], group[1:]
    for other in surrounding:
        fact(border_rel, main_territory, other)

# Создание переменных для запросов
query_var_1 = var()
query_var_2 = var()

# Проверка границы между Колорадо и Южной Каролиной
answer = run(0, query_var_1, border_rel('Colorado', 'South Carolina'))
print('\nИмеют ли общую границу Колорадо и Южная Каролина?:')
print('Да' if len(answer) else 'Нет')

# Территории, граничащие с Аризоной
answer = run(0, query_var_1, border_rel('Arizona', query_var_1))
print('\nСписок территорий, граничащих с Аризоной:')
for item in answer:
    print(item)

# Морские штаты, соседние с Алабамой
answer = run(0, query_var_1, border_rel('Alabama', query_var_1), seaside_rel(query_var_1))
print('\nМорские штаты, граничащие с Алабамой:')
for item in answer:
    print(item)

# Выбор N территорий, имеющих выход к морскому штату
N = 5
answer = run(N, query_var_1, seaside_rel(query_var_2), border_rel(query_var_1, query_var_2))
print('\nПервые ' + str(N) + ' территорий, соседствующих с морским штатом:')
for item in answer:
    print(item)

# Территории, граничащие одновременно с Техасом и Нью-Мексико
answer = run(0, query_var_1, border_rel('Texas', query_var_1), border_rel('New Mexico', query_var_1))
print('\nТерритории, граничащие и с Техасом, и с Нью-Мексико:')
for item in answer:
    print(item)


Имеют ли общую границу Колорадо и Южная Каролина?:
Нет

Список территорий, граничащих с Аризоной:
Colorado
California
New Mexico
Utah
Nevada

Морские штаты, граничащие с Алабамой:
Georgia
Mississippi
Florida

Первые 5 территорий, соседствующих с морским штатом:
Texas
Pennsylvania
Alabama
Massachusetts
Arkansas

Территории, граничащие и с Техасом, и с Нью-Мексико:
Oklahoma


Создание решателя головоломок

In [5]:
from kanren import *
from kanren.core import lall


# Объявление переменной
residents = var()


# Определение правил
rules = lall(
    # Число людей -4
    (eq, (var(), var(), var(), var()), residents),

    # У Ивана есть автомобиль синего цвета
    (membero, ('Иван', var(), 'blue', var()), residents),

    # Человек, у которого есть кот, живет в Канаде
    (membero, (var(), 'cat', var(), 'Canada'), residents),

    # Матвей живет в США
    (membero, ('Матвей', var(), var(), 'USA'), residents),

    # Человек, у которого есть автомобиль черного цвета, живет в Австралии
    (membero, (var(), var(), 'black', 'Australia'), residents),

    # У Жака есть кот
    (membero, ('Жак', 'cat', var(), var()), residents),

    # Альфред живет в Австралии
    (membero, ('Альфред', var(), var(), 'Australia'), residents),

    # Человек, у которого есть собака, живет во Франции
    (membero, (var(), 'dog', var(), 'France'), residents),

    # У кого есть кролик?
    (membero, (var(), 'rabbit', var(), var()), residents)
)


# Запуск решателя
solutions = run(0, residents, rules)


# Выполнение решателя
result = [house for house in solutions[0] if 'rabbit' in house][0][0]


# Извлечение результата
print('\n' + result + ' является владельцем кролика')
print('\nВот все детали:')
attributes = ['Имя', 'Питомец', 'Цвет', 'Страна']
print('\n' + '\t\t'.join(attributes))
print('=' * 57)
for item in solutions[0]:
    print('')
    print('\t\t'.join([str(x) for x in item]))


Матвей является владельцем кролика

Вот все детали:

Имя		Питомец		Цвет		Страна

Иван		dog		blue		France

Жак		cat		~_125		Canada

Матвей		rabbit		~_127		USA

Альфред		~_129		black		Australia
